In [1]:
import re
import ast
import pyspark
import time
import datetime
import html
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
#visualisation
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.charts import Histogram, Scatter, Bar, TimeSeries, show, \
                         output_file, output_notebook, color, marker, defaults
from bokeh.layouts import column
from bokeh.models import NumeralTickFormatter
import pandas as pd

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [3]:
# Domyślny rozmiar wykresów Bokeh:
defaults.width = 900
defaults.height = 500

In [4]:
# Funkcja pomocnicza: wyszukiwanie podanego atrybutu w linii pliku XML,
# zwraca wartość znalezionego atrybutu lub None jeśli atrybut nie istnieje

def attribute_search(attribute, string):
    result = re.search(attribute + '=\"(.*?)\"', string)
    if result:
        return result.group(1).replace('"', '')
    else:
        return None

In [5]:
# Funkcje pomocnicze: interpretacja plików XML (różne schematy danych)
# ze Stackoverflow

def tags_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['TagName'] = attribute_search('TagName', c)
    count = attribute_search('Count', c)
    row['Count'] = int(count) if count else None
    excerpt = attribute_search('ExcerptPostId', c)
    row['ExcerptPostId'] = int(excerpt) if excerpt else None
    wiki = attribute_search('WikiPostId', c)
    row['WikiPostId'] = int(wiki) if wiki else None
    return pyspark.Row(**row)

def badges_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['UserId'] = int(attribute_search('UserId', c))
    row['Name'] = attribute_search('Name', c)
    row['Date'] = datetime.datetime.strptime(attribute_search('Date', c), "%Y-%m-%dT%H:%M:%S.%f")
    row['Class'] = int(attribute_search('Class', c))
    row['TagBased'] = ast.literal_eval(attribute_search('TagBased', c))
    return pyspark.Row(**row)

def users_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['Reputation'] = int(attribute_search('Reputation', c))
    row['CreationDate'] = datetime.datetime.strptime(attribute_search('CreationDate', c), "%Y-%m-%dT%H:%M:%S.%f")
    row['DisplayName'] = attribute_search('DisplayName', c)
    row['LastAccessDate'] = datetime.datetime.strptime(attribute_search('LastAccessDate', c), "%Y-%m-%dT%H:%M:%S.%f")
    row['WebsiteUrl'] = attribute_search('WebsiteUrl', c)
    row['Location'] = attribute_search('Location', c)
    age = attribute_search('Age', c)
    row['Age'] = int(age) if age else None
    row['Views'] = int(attribute_search('Views', c))
    row['UpVotes'] = int(attribute_search('UpVotes', c))
    row['DownVotes'] = int(attribute_search('DownVotes', c))
    return pyspark.Row(**row)

def posts_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['PostTypeId'] = int(attribute_search('PostTypeId', c))
    found_id = attribute_search('ParentId', c)
    row['ParentId'] = int(found_id) if found_id else None
    found_id = attribute_search('AcceptedAnswerId', c)
    row['AcceptedAnswerId'] = int(found_id) if found_id else None
    row['CreationDate'] = datetime.datetime.strptime(attribute_search('CreationDate', c), "%Y-%m-%dT%H:%M:%S.%f")
    row['Score'] = int(attribute_search('Score', c))
    vc = attribute_search('ViewCount', c)
    row['ViewCount'] = int(vc) if vc else None
    owner = attribute_search('OwnerUserId', c)
    row['OwnerUserId'] = int(owner) if owner else None
    lasted = attribute_search('LastEditorUserId', c)
    row['LastEditorUserId'] = int(lasted) if lasted else None

    row['Body'] = re.sub('(<!--.*?-->|<[^>]*>)', '', html.unescape(attribute_search('Body', c)))
    title = attribute_search('Title', c)
    row['Title'] = title if title else None
    tags = attribute_search('Tags', c)
    row['Tags'] = html.unescape(tags).replace('<', '').replace('>', ' ') if tags else None
    date = attribute_search('ClosedDate', c)
    row['ClosedDate'] = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S.%f") if date else None

    count = attribute_search('AnswerCount', c)
    row['AnswerCount'] = int(count) if count else None
    count = attribute_search('CommentCount', c)
    row['CommentCount'] = int(count) if count else None
    count = attribute_search('FavoriteCount', c)
    row['FavoriteCount'] = int(count) if count else None
        
    return pyspark.Row(**row)

def comments_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['PostId'] = int(attribute_search('PostId', c))
    row['Score'] = int(attribute_search('Score', c))
    row['Text'] = re.sub('(<!--.*?-->|<[^>]*>)', '', html.unescape(attribute_search('Text', c)))
    row['CreationDate'] = datetime.datetime.strptime(attribute_search('CreationDate', c), "%Y-%m-%dT%H:%M:%S.%f")
    user = attribute_search('UserId', c)
    row['UserId'] = int(user) if user else None
    return pyspark.Row(**row)

def post_history_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['PostHistoryTypeId'] = int(attribute_search('PostHistoryTypeId', c))
    row['PostId'] = int(attribute_search('PostId', c))
    comm = attribute_search('Comment', c)
    row['Comment'] = comm if comm else None
    text = attribute_search('Text', c)
    row['Text'] = re.sub('(<!--.*?-->|<[^>]*>)', '', html.unescape(text)) if text else None
    return pyspark.Row(**row)

def post_links_from_xml(line):
    c = line.replace('<row', '').replace('/>', '')
    row = dict()
    row['Id'] = int(attribute_search('Id', c))
    row['CreationDate'] = datetime.datetime.strptime(attribute_search('CreationDate', c), "%Y-%m-%dT%H:%M:%S.%f")
    row['PostId'] = int(attribute_search('PostId', c))
    row['RelatedPostId'] = int(attribute_search('RelatedPostId', c))
    row['LinkTypeId'] = int(attribute_search('LinkTypeId', c))
    return pyspark.Row(**row)

In [6]:
# Wczytanie danych do RDD z plików XML, a następnie konwersja RDD do DF: 
xml_load_path = 'file:///home/marek/Dokumenty/Notebooks/bd/gis_stack_spark/data/'

# Słownik: nazwa pliku i odpowiadająca mu funkcja pomocnicza interpretująca
# schemat danych w pliku XML
xml_load_list = {'Tags.xml': tags_from_xml, 'Badges.xml': badges_from_xml, \
                 'Users.xml': users_from_xml,'Posts.xml': posts_from_xml, \
                 'Comments.xml': comments_from_xml,'PostHistory.xml': post_history_from_xml, \
                 'PostLinks.xml': post_links_from_xml}

tags_rdd = sc.textFile(xml_load_path + 'Tags.xml').filter(lambda line: "row" in line) \
             .map(lambda l: xml_load_list['Tags.xml'](l))

badges_rdd = sc.textFile(xml_load_path + 'Badges.xml').filter(lambda line: "row" in line) \
               .map(lambda l: xml_load_list['Badges.xml'](l))

users_rdd = sc.textFile(xml_load_path + 'Users.xml').filter(lambda line: "row" in line) \
              .map(lambda l: xml_load_list['Users.xml'](l))

posts_rdd = sc.textFile(xml_load_path + 'Posts.xml').filter(lambda line: "row" in line) \
              .map(lambda l: xml_load_list['Posts.xml'](l))

comments_rdd = sc.textFile(xml_load_path + 'Comments.xml').filter(lambda line: "row" in line) \
                 .map(lambda l: xml_load_list['Comments.xml'](l))

post_history_rdd = sc.textFile(xml_load_path + 'PostHistory.xml').filter(lambda line: "row" in line) \
                     .map(lambda l: xml_load_list['PostHistory.xml'](l))

post_links_rdd = sc.textFile(xml_load_path + 'PostLinks.xml').filter(lambda line: "row" in line) \
                   .map(lambda l: xml_load_list['PostLinks.xml'](l))

# Konwersja na DataFrame:
users = sqlContext.createDataFrame(users_rdd)
badges = sqlContext.createDataFrame(badges_rdd)
posts = sqlContext.createDataFrame(posts_rdd)
tags = sqlContext.createDataFrame(tags_rdd)
comments = sqlContext.createDataFrame(comments_rdd)
post_history = sqlContext.createDataFrame(post_history_rdd)
post_links = sqlContext.createDataFrame(post_links_rdd)

In [7]:
# Liczba użytowników (oś Y) vs liczba postów (oś X):
# DF 'posts' kolumna OwnerUserId = DF 'users' kolumna Id --> można wyjąć np. DisplayName użytkownika
# do samego policzenia: groupby OwnerUserId w tabeli posts

users_by_posts = posts.groupBy('OwnerUserId').count()\
                      .select(col('OwnerUserId').alias('user'), col('count').alias('number_posts'))\
                      .groupBy('number_posts').count()\
                      .select(col('number_posts'), col('count').alias('number_users'))\
                      .orderBy('number_posts')

In [8]:
# Wersja z RDD:
# x = users_by_posts.rdd.map(lambda x: x.number_posts).collect()
# y = users_by_posts.rdd.map(lambda y: y.number_users).collect()

# lub z pandas DF:
x = users_by_posts.toPandas()['number_posts'].tolist()
y = users_by_posts.toPandas()['number_users'].tolist()

In [9]:
# Użycie Bokeh do wyświetlenia danych
output_notebook()
plot = figure(plot_width=900, plot_height=500,\
              x_axis_type="log", y_axis_type="log",\
              x_axis_label="Liczba postów", y_axis_label="Liczba użytkowników")
plot.circle(x, y, size=6, color="green", alpha=0.4)
show(plot)

Loading BokehJS ...

In [ ]:
# Aktywność community GIS: Number of Questions vs Answers
# posts -- PostTypeId = 1 (Question)
# --> AnswerCount

questions_vs_answers = posts.select(col('Id'), col('AnswerCount')).where(posts.PostTypeId == 1)\
                            .groupBy('AnswerCount').count()\
                            .select(col('AnswerCount').alias('answers'), col('count').alias('number_questions'))\
                            .orderBy('answers')

In [ ]:
# wersja zamiast .toPandas() -- tworzy listę
#x = questions_vs_answers.rdd.map(lambda x: x.answers).collect()
#top = questions_vs_answers.rdd.map(lambda y: y.number_questions).collect()

In [ ]:
# wykres finalny
output_notebook()
plot = Bar(questions_vs_answers.toPandas(), 'answers', values='number_questions', \
           title="Odpowiedzi vs liczba pytań", legend='top_right', color="navy", \
           ylabel='Pytania', xlabel='Odpowiedzi')

show(plot)

In [ ]:
total_questions = posts.select(posts.Id).where(posts.PostTypeId == 1).count()

unanswered_questions = questions_vs_answers.select(col('number_questions')) \
                       .where(questions_vs_answers.answers == 0) \
                       .collect()[0].number_questions

In [ ]:
print("Liczba pytań na forum GIS:    {}".format(total_questions))
print("Liczba pytań bez odpowiedzi:  {}".format(unanswered_questions))
print("Procent pytań bez odpowiedzi: {:.2%}".format(unanswered_questions/total_questions))

In [ ]:
posts.printSchema()

In [ ]:
tags.printSchema()

In [14]:
# Najpopularniejsze tagi na forum
popular_tags = tags.select(tags.TagName, tags.Count)\
                   .sort(tags.Count, ascending=False).toPandas()

In [15]:
popular_tags.head(40)

,TagName,Count
0,qgis,16285
1,arcgis-desktop,14326
2,arcpy,5919
3,python,5678
4,postgis,4967
5,raster,4432
6,arcgis-10.0,3528
7,arcmap,3509
8,geoserver,3308
9,openlayers-2,3270


In [16]:
output_notebook()
plot = Bar(popular_tags.head(20), 'TagName', values='Count', \
           title="Najpopularniejsze tagi na forum GIS", legend=False, color="darkviolet", \
           ylabel='Liczba postów', xlabel='Tag (nazwa)')
show(plot)

Loading BokehJS ...

In [ ]:
# Wstępna weryfikacja danych / zapytań

In [ ]:
posts.filter(posts.Tags.rlike(r'\bpostgis\b')).count()

In [ ]:
posts.count()

In [ ]:
# Zapytanie dla jednego produktu GIS (tu tag 'arcgis-desktop'):
gis_posts_data = posts.filter(posts.Tags.rlike(r'\barcgis-desktop\b')) \
    .select(concat_ws('.', year("CreationDate"), \
                           lpad(month("CreationDate"), 2, "0")).alias('Months')) \
    .groupBy('Months').count().sort('Months').toPandas()

gis_posts_data.rename(columns={'count': 'ArcGIS'}, inplace=True)

# Można dodać kolejną kolumnę z danymi do DF:
# arcgis_desktop['PostGIS'] = postgis['PostGIS']

In [ ]:
# Dla kilku produktów: iterujemy po liście tagów i składamy kolejne kolumny w DF pandas

In [ ]:
gis_products = ['qgis', 'arcgis-desktop', 'postgis', 'openstreetmap', 'grass', 'google-maps']
# gis_percentages = ['%{0}'.format(i) for i in gis_products]

In [ ]:
# Analiza dla produktów ArcGIS (różne wersje w tagach)
gis_products = ['arcgis-desktop', 'arcgis-server', 'arcgis-10.0', \
                'arcgis-10.1', 'arcgis-10.2', 'arcgis-10.3' ,'arcgis-javascript-api']

In [ ]:
# Wyliczamy totals (dla wszystkich postow) w gis_posts_data.
# Dzięki temu mamy DF do którego potem doklejamy kolumny
# z wynikami dla innych tagów.
print('Wyliczam statystyki dla wszystkich postow.')
gis_posts_data = posts.filter(posts.Tags.isNotNull()).select(concat_ws('.', year("CreationDate"), \
                              lpad(month("CreationDate"), 2, "0")).alias('Months')) \
                      .groupBy('Months').count().sort('Months').toPandas()

gis_posts_data.rename(columns={'count': 'wszystkie'}, inplace=True)

for gis_p in gis_products:
    print('Wyliczam posty dla produktu: {0}'.format(gis_p))
    product_posts = posts.filter(posts.Tags.isNotNull() & posts.Tags.rlike(r'\b{0}\b'.format(gis_p))) \
        .select(concat_ws('.', year("CreationDate"), lpad(month("CreationDate"), 2, "0")).alias('Months')) \
        .groupBy('Months').count().sort('Months').toPandas()
    product_posts.rename(columns={'count': gis_p}, inplace=True)
    gis_posts_data = pd.merge(gis_posts_data, product_posts, on='Months', how='outer')

print('Gotowe.')

In [ ]:
product_posts

In [ ]:
gis_posts_data

In [ ]:
# Test funkcji .rolling:
gis_posts_data.rolling(on='Months', window=5).mean()

In [ ]:
# Obliczanie % postow dla produktu ws totals
# np. dla produktu qgis obliczamy qgis_p jako
# (qgis / wszystkie) * 100%
for gis_p in gis_products:
    gis_posts_data['%' + gis_p] = (gis_posts_data[gis_p].fillna(0) \
                                   / gis_posts_data.wszystkie) * 100

In [ ]:
gis_posts_data

In [ ]:
# Trend czasowy -- posty na miesiąc
output_notebook()

#tsline = TimeSeries(gis_posts_data, x='Months', y=['wszystkie'] + gis_products, \
#         title="Posty na miesiąc", xlabel='Data (miesiące)', ylabel='Produkty GIS', legend=True)
tsline = TimeSeries(gis_posts_data, x='Months', y=gis_products, \
         title="Posty na miesiąc (liczba)", xlabel='Data (miesiące)', ylabel='Produkty GIS', legend=True)

show(column(tsline))

In [ ]:
# Trend czasowy wygładzony -- posty na miesiąc
output_notebook()

#tsline = TimeSeries(gis_posts_data, x='Months', y=['wszystkie'] + gis_products, \
#         title="Posty na miesiąc", xlabel='Data (miesiące)', ylabel='Produkty GIS', legend=True)
tsline = TimeSeries(gis_posts_data.rolling(on='Months', window=4).mean(), x='Months', y=gis_products, \
         title="Posty na miesiąc (liczba), wygładzone", \
         xlabel='Data (miesiące)', ylabel='Produkty GIS', legend=True)

show(column(tsline))

In [ ]:
# Trend czasowy -- posty na miesiąc w % do całości
output_notebook()

tsline = TimeSeries(gis_posts_data, x='Months', y=['%{0}'.format(i) for i in gis_products], \
         title="Posty na miesiąc (%całosci)", xlabel='Data (miesiące)', ylabel='Produkty GIS', legend=True)
#tsline.xaxis.minor_tick_in = -3
#tsline.xaxis.minor_tick_out = 2
#tsline.xaxis.major_tick_out = 3

#tsline.xgrid.grid_line_color = None
tsline.ygrid.grid_line_dash = [6, 4]

show(column(tsline))

In [ ]:
# Ja, jako użytkownik forum, chcę wiedzieć, jak długo zajmie uzyskanie
# dobrej odpowiedzi dla danego produktu (lista produktów).
#
# ile czasu upływa od zadania pytania
# do uzyskania zaakceptowanej odpowiedzi (dla tagów / produktów typu ArcGIS -- wykres??).

In [ ]:
# Wyznaczenie czasu do zaakceptowanej odpowiedzi dla produktu
# najpierw filter
# potem oblicz



In [10]:
# Czas odpowiedzi
# posts -- PostTypeId = 1 (Question), 2 (Answer)

questions = posts.select(posts.Id, posts.AcceptedAnswerId,\
                         posts.CreationDate.alias('QuestionDate'),\
                         posts.Tags).where(posts.PostTypeId == 1)

# alias('Parent') to obejście problemu
# org.apache.spark.sql.AnalysisException:
# resolved attribute(s) ID missing from ID in operator !Join
# https://issues.apache.org/jira/browse/SPARK-10925
answers = posts.select(posts.Id.alias('AnswerId'), posts.ParentId.alias('Parent'),\
                       posts.CreationDate.alias('AnswerDate')).where(posts.PostTypeId == 2)

In [11]:
questions.printSchema()
answers.printSchema()

root
 |-- Id: long (nullable = true)
 |-- AcceptedAnswerId: long (nullable = true)
 |-- QuestionDate: timestamp (nullable = true)
 |-- Tags: string (nullable = true)

root
 |-- AnswerId: long (nullable = true)
 |-- Parent: long (nullable = true)
 |-- AnswerDate: timestamp (nullable = true)



In [12]:
# UDF do obliczenia różnicy czasu (w sekundach)
# x, y -- datetime
def time_delta(x, y):
    delta = int((y - x).total_seconds())
    return delta

time_delta_udf = udf(time_delta, returnType = IntegerType())

In [14]:
# join jest dla wszystkich odpowiedzi (a nie tylko AcceptedAnswer)
joined = questions.join(answers, questions.Id == answers.Parent)
#         .select(questions.Id, questions.AcceptedAnswerId, questions.CreationDate, answers.CreationDate)

# dla zaakceptowanych trzeba użyć filtr AcceptedAnswerId.isNotNull()
#df = questions.join(answers, questions.Id == answers.Parent).filter(col('AcceptedAnswerId').isNotNull()).count()

In [ ]:
# Posty "merged" mają często zamienione oryginalne pytanie z późniejszym,
# co sprawia, że czas odpowiedzi jest przed czasem zadania pytania.
# Przykład:
# https://gis.stackexchange.com/questions/124794/installing-qgis-with-ecw-support-on-mac
# w dumpie z 14 marca ma 2 dodatkowe linki (merge)
# trzeba znaleźć posty, które w post_history mają PostHistoryTypeId == 37 lub == 38

In [20]:
post_history.printSchema()

root
 |-- Comment: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- PostHistoryTypeId: long (nullable = true)
 |-- PostId: long (nullable = true)
 |-- Text: string (nullable = true)



In [16]:
# Liczba merged_posts źródłowych i docelowych.
# Posty mające w historii (tabela post_history):
# PostHistoryTypeId
# 37. Post merge source
# 38. Post merge destination
post_history.where((post_history.PostHistoryTypeId == 37) | (post_history.PostHistoryTypeId == 38)).count()

665

In [ ]:
# (Za dużo otwartych plików)
# questions.join(post_history, questions.Id == post_history.PostId).count()

In [ ]:
# (Za dużo otwartych plików)
# posts.join(post_history, posts.Id == post_history.PostId).first()

#                            .filter(col('AcceptedAnswerId').isNotNull())\


In [68]:
# ze zbioru postów należy usunąć posty: zmerdżowane (merged)
# oraz duplikaty (duplicate). Merged mają często ujemne czasy
# odpowiedzi, duplikaty -- bardzo długie czasy (jeśli np. nowy
# post jest duplikatem starego postu)
merged_posts_ids = post_history.select(post_history.PostId) \
                   .where((post_history.PostHistoryTypeId == 37) | \
                          (post_history.PostHistoryTypeId == 38))

duplicate_posts_ids = post_links.select(post_links.PostId).where(post_links.LinkTypeId == 3)

remove_ids = merged_posts_ids.union(duplicate_posts_ids).distinct()

In [67]:
remove_ids.count()

4463

In [65]:
posts.count()

179507

In [69]:
#  anti_join using a dataframe:
#    tbl1.as("a").join(tbl2.as("b"), $"a.id" === $"b.id", "left_anti")
# This PR provides serves as the basis for implementing `NOT EXISTS` and `NOT IN 
# (...)` correlated sub-queries. It would also serve as good basis for 
# implementing an more efficient `EXCEPT` operator.
posts_nd = posts.join(remove_ids, posts.Id == remove_ids.PostId, 'left_anti')

In [70]:
posts_nd.count()

176521

In [71]:
questions = posts_nd.select(posts_nd.Id, posts_nd.AcceptedAnswerId,\
                         posts_nd.CreationDate.alias('QuestionDate'),\
                         posts_nd.Tags).where(posts_nd.PostTypeId == 1)

In [72]:
questions.count()

76472

In [17]:
questions.count()

79194

In [22]:
answers.count()

96244

In [15]:
merged_posts_ids.count()

665

In [16]:
#  anti_join using a dataframe:
#    tbl1.as("a").join(tbl2.as("b"), $"a.id" === $"b.id", "left_anti")
# This PR provides serves as the basis for implementing `NOT EXISTS` and `NOT IN 
# (...)` correlated sub-queries. It would also serve as good basis for 
# implementing an more efficient `EXCEPT` operator.

questions_nm = questions.join(merged_posts_ids, questions.Id == merged_posts_ids.PostId, 'left_anti')

In [17]:
answers_nm = answers.join(merged_posts_ids, answers.AnswerId == merged_posts_ids.PostId, 'left_anti')

In [18]:
questions_nm.count()

78813

In [63]:
# prosty test po joinie. juz nie mamy df joined (do usuniecia) -- chodzi o pokazanie zasady
# joined_timedelta = joined.withColumn('TimeDelta', time_delta_udf(joined.QuestionDate, joined.AnswerDate))

In [30]:
# tylko z zaakceptowanymi odpowiedziami (ale uwzględnia pozostałe odpowiedzi dla pytania)
joined_timedelta = \
    questions_nm.join(answers_nm, questions_nm.Id == answers_nm.Parent) \
    .filter(col('AcceptedAnswerId').isNotNull()) \
    .withColumn('TimeDelta', time_delta_udf(questions_nm.QuestionDate, answers_nm.AnswerDate))

In [32]:
# tylko dla zaakceptowanych odpowiedzi
joined_timedelta = \
    questions_nm.join(answers_nm, questions_nm.Id == answers_nm.Parent) \
    .filter(col('AcceptedAnswerId').isNotNull() & (col('AcceptedAnswerId') == col('AnswerId'))) \
    .withColumn('TimeDelta', time_delta_udf(questions_nm.QuestionDate, answers_nm.AnswerDate))

In [16]:
# wszystkie z odpowiedziami (nie tylko zaakceptowanymi)
joined_timedelta = questions_nm.join(answers_nm, questions_nm.Id == answers_nm.Parent) \
                               .withColumn('TimeDelta', time_delta_udf(questions_nm.QuestionDate, answers_nm.AnswerDate))

In [20]:
joined_timedelta.printSchema()

root
 |-- Id: long (nullable = true)
 |-- AcceptedAnswerId: long (nullable = true)
 |-- QuestionDate: timestamp (nullable = true)
 |-- Tags: string (nullable = true)
 |-- AnswerId: long (nullable = true)
 |-- Parent: long (nullable = true)
 |-- AnswerDate: timestamp (nullable = true)
 |-- TimeDelta: integer (nullable = true)



In [44]:
joined_timedelta.count()

31251

In [43]:
joined_timedelta.where(joined_timedelta.TimeDelta > 0).count()

31165

In [34]:
joined_timedelta.collect()

[Row(Id=711, AcceptedAnswerId=720, QuestionDate=datetime.datetime(2010, 8, 3, 23, 35, 42, 803000), Tags='projection understanding area ', AnswerId=720, Parent=711, AnswerDate=datetime.datetime(2010, 8, 4, 4, 28, 53, 343000), TimeDelta=17590),
 Row(Id=172, AcceptedAnswerId=965, QuestionDate=datetime.datetime(2010, 7, 23, 8, 44, 30, 317000), Tags='data ordnance-survey united-kingdom ', AnswerId=965, Parent=172, AnswerDate=datetime.datetime(2010, 8, 8, 22, 46, 14, 297000), TimeDelta=1432903),
 Row(Id=1079, AcceptedAnswerId=1085, QuestionDate=datetime.datetime(2010, 8, 11, 18, 6, 51, 600000), Tags='arcgis-desktop raster vector dem esri-production-mapping ', AnswerId=1085, Parent=1079, AnswerDate=datetime.datetime(2010, 8, 11, 19, 37, 53, 687000), TimeDelta=5462),
 Row(Id=2013, AcceptedAnswerId=2014, QuestionDate=datetime.datetime(2010, 9, 19, 16, 50, 16, 397000), Tags='google-maps kml ', AnswerId=2014, Parent=2013, AnswerDate=datetime.datetime(2010, 9, 19, 17, 25, 55, 487000), TimeDelta=21

In [41]:
quantiles = [0.0, 0.25, 0.5, 0.75, 1.0]
qlabels = ["q"+str(int(q * 100)) for q in quantiles]

In [42]:
joined_timedelta.approxQuantile('TimeDelta', quantiles, 0)

[0.0, 2870.0, 12985.0, 85048.0, 175361051.0]

In [45]:
joined_timedelta.where(joined_timedelta.TimeDelta > 0).approxQuantile('TimeDelta', quantiles, 0)

[18.0, 2901.0, 13074.0, 85394.0, 175361051.0]

In [48]:
joined_timedelta.where((joined_timedelta.TimeDelta > 0) & posts.Tags.rlike(r'\b{0}\b'.format('qgis'))) \
                       .approxQuantile('TimeDelta', quantiles, 0)

[71.0, 2697.0, 12175.0, 77715.0, 175361051.0]

In [50]:
joined_timedelta.where((joined_timedelta.TimeDelta > 0) & posts.Tags.rlike(r'\b{0}\b'.format('qgis')))\
                .sort(joined_timedelta.TimeDelta, ascending=False).collect()

[Row(Id=7078, AcceptedAnswerId=212483, QuestionDate=datetime.datetime(2011, 3, 10, 23, 14, 27, 330000), Tags='qgis pdf geospatial-pdf ', AnswerId=212483, Parent=7078, AnswerDate=datetime.datetime(2016, 9, 29, 14, 38, 38, 640000), TimeDelta=175361051),
 Row(Id=60880, AcceptedAnswerId=207408, QuestionDate=datetime.datetime(2013, 5, 15, 22, 11, 48, 943000), Tags='qgis mapbox qgis-openlayers-plugin vector-tiles mapbox-studio ', AnswerId=207408, Parent=60880, AnswerDate=datetime.datetime(2016, 8, 18, 9, 37, 52, 927000), TimeDelta=102857163),
 Row(Id=28870, AcceptedAnswerId=162876, QuestionDate=datetime.datetime(2012, 7, 4, 16, 19, 53, 550000), Tags='qgis topography ', AnswerId=162876, Parent=28870, AnswerDate=datetime.datetime(2015, 9, 14, 16, 56, 28, 857000), TimeDelta=100830995),
 Row(Id=40760, AcceptedAnswerId=176095, QuestionDate=datetime.datetime(2012, 11, 9, 17, 46, 9, 813000), Tags='qgis style legend ', AnswerId=176095, Parent=40760, AnswerDate=datetime.datetime(2016, 1, 8, 16, 9, 26

In [51]:
posts.where(posts.Id == 7078).collect()

[Row(AcceptedAnswerId=212483, AnswerCount=2, Body="In ArcMap, when you export data to PDF it preserves the layering within the PDF so that you can turn on/off individual map layers (within the PDF itself).  When I export from QGIS to PDF, I don't appear to have the option for this.\n\nDoes QGIS support preserving layers when exporting to PDF?  Is there a special setting needed to enable this capability?  Or has anyone wrote a plugin for this feature?\n", ClosedDate=None, CommentCount=1, CreationDate=datetime.datetime(2011, 3, 10, 23, 14, 27, 330000), FavoriteCount=1, Id=7078, LastEditorUserId=28067, OwnerUserId=1072, ParentId=None, PostTypeId=1, Score=21, Tags='qgis pdf geospatial-pdf ', Title='Can QGIS preserve layers when exporting to PDF?', ViewCount=2960)]

In [52]:
post_history.where(post_history.PostId == 7087).collect()

[Row(Comment=None, Id=15377, PostHistoryTypeId=2, PostId=7087, Text='i Created topology graph of a polygon feature and i copied geometry of topology edges into line feature class,and topology nodes into point feature class\r\n\r\ni created topology layer containing featureclasses as polygon,line,point .\r\n\r\nusing IEnumtopoparent i am getting FID of individual points ,lines.\r\n \r\ni want to get polygon arclist,for getting this \r\nusing topologyGRAPH.getparentedges(polygon feature class,OID) i am accessing individual edges of polygon. using topoparent.m_FID iam getting fid of parents. but iam not getting line feature as topoparent\r\n\r\nto get line feature as topoparent for tpology edges what topology rules should I applY \r\nwhen creating topology layer\r\n\r\npls helpe me.....\r\n'),
 Row(Comment=None, Id=15378, PostHistoryTypeId=1, PostId=7087, Text='topology rules for structuring topology data to get edge FID in topology graph'),
 Row(Comment=None, Id=15379, PostHistoryTypeId=

[Row(PostId=957),
 Row(PostId=1498),
 Row(PostId=3633),
 Row(PostId=3849),
 Row(PostId=4531),
 Row(PostId=5191),
 Row(PostId=6296),
 Row(PostId=6470),
 Row(PostId=6508),
 Row(PostId=6968),
 Row(PostId=6990),
 Row(PostId=7274),
 Row(PostId=7275),
 Row(PostId=7651),
 Row(PostId=8062),
 Row(PostId=8238),
 Row(PostId=8288),
 Row(PostId=8414),
 Row(PostId=8449),
 Row(PostId=9304),
 Row(PostId=9678),
 Row(PostId=10423),
 Row(PostId=10619),
 Row(PostId=10625),
 Row(PostId=10701),
 Row(PostId=10756),
 Row(PostId=11015),
 Row(PostId=11223),
 Row(PostId=11420),
 Row(PostId=11431),
 Row(PostId=11761),
 Row(PostId=11898),
 Row(PostId=12305),
 Row(PostId=12630),
 Row(PostId=12860),
 Row(PostId=13004),
 Row(PostId=13107),
 Row(PostId=13686),
 Row(PostId=13920),
 Row(PostId=13975),
 Row(PostId=14007),
 Row(PostId=14131),
 Row(PostId=14133),
 Row(PostId=14142),
 Row(PostId=14438),
 Row(PostId=14446),
 Row(PostId=14469),
 Row(PostId=14650),
 Row(PostId=14789),
 Row(PostId=14903),
 Row(PostId=14993),
 R

In [17]:
# df.groupBy('column').count().rdd.values().histogram()

hist_v = joined_timedelta.groupBy(joined_timedelta.TimeDelta).count().rdd.values().histogram(40)

In [21]:
count_timedelta = joined_timedelta.groupBy(joined_timedelta.TimeDelta).count() \
                         .select(col('TimeDelta'), col('count').alias('Count'))

In [23]:
count_timedelta.toPandas()

,TimeDelta,Count
0,2122,4
1,1238,4
2,3997,3
3,77803,1
4,4818,4
5,879114,1
6,15846,1
7,22346,1
8,28577,1
9,91299,1


In [18]:
hist_v.printSchema()

AttributeError: 'tuple' object has no attribute 'printSchema'

In [102]:
hist_v.where(hist_v.Count > 1).sort('Count', ascending=False).collect()

[Row(TimeDelta=0, Count=96),
 Row(TimeDelta=815, Count=15),
 Row(TimeDelta=1000, Count=14),
 Row(TimeDelta=591, Count=14),
 Row(TimeDelta=853, Count=13),
 Row(TimeDelta=1378, Count=13),
 Row(TimeDelta=620, Count=13),
 Row(TimeDelta=749, Count=13),
 Row(TimeDelta=1156, Count=12),
 Row(TimeDelta=2240, Count=12),
 Row(TimeDelta=699, Count=12),
 Row(TimeDelta=1369, Count=12),
 Row(TimeDelta=1466, Count=11),
 Row(TimeDelta=544, Count=11),
 Row(TimeDelta=2418, Count=11),
 Row(TimeDelta=644, Count=11),
 Row(TimeDelta=1091, Count=11),
 Row(TimeDelta=843, Count=11),
 Row(TimeDelta=489, Count=11),
 Row(TimeDelta=870, Count=11),
 Row(TimeDelta=1015, Count=11),
 Row(TimeDelta=956, Count=11),
 Row(TimeDelta=795, Count=11),
 Row(TimeDelta=893, Count=11),
 Row(TimeDelta=782, Count=11),
 Row(TimeDelta=802, Count=11),
 Row(TimeDelta=1411, Count=11),
 Row(TimeDelta=1246, Count=11),
 Row(TimeDelta=1359, Count=11),
 Row(TimeDelta=458, Count=10),
 Row(TimeDelta=898, Count=10),
 Row(TimeDelta=321, Count=10)

In [19]:
hist_df = pd.DataFrame(list(zip(list(hist_v)[0], \
                       list(hist_v)[1])), \
                       columns=['bin','frequency'])

In [20]:
hist_df

,bin,frequency
0,1.000,32901
1,3.375,1314
2,5.750,601
3,8.125,81
4,10.500,21
5,12.875,7
6,15.250,0
7,17.625,0
8,20.000,0
9,22.375,0


In [93]:
output_notebook()

plot = Bar(hist_df, 'bin', values='frequency', \
           title="Histogram", legend=False, color="darkviolet", \
           ylabel='Freq', xlabel='Bin')
show(plot)

Loading BokehJS ...

In [36]:
# .min() -- wyszedł problem ze zmergowanymi postami
# różnica czasu jest ujemna (!!! trzeba odfiltrować merged_posts!!!)
joined_timedelta.select(joined_timedelta.TimeDelta).groupBy().mean().collect()

[Row(avg(TimeDelta)=5619135.498808069)]

In [35]:
joined_timedelta.where(joined_timedelta.TimeDelta == 0).count()

96

In [32]:
posts.where(posts.Id == 203380).collect()

[Row(AcceptedAnswerId=203381, AnswerCount=1, Body="My ArcGIS Pro install recently updated from 1.2 to 1.3.  \n\nIn reading Python in ArcGIS Pro 1.3 I discovered that Esri have changed how ArcGIS Pro uses Python/ArcPy.  \n\n\n  Legacy:\n  In ArcGIS Pro 1.0 through 1.2, if you needed to use Python\n  outside of ArcGIS Pro, you also had to install Python for ArcGIS Pro.\n  With changes at 1.3, this is no longer required. To avoid confusion,\n  uninstall Python for ArcGIS Pro.\n\n\nI uninstalled Python for ArcGIS Pro, but now I can't get my IDE (Visual Studio 2013) to see arcpy anymore.  In fact I don't think it's seeing Python 3 at all (switching to Python 2.7 still works)\n\nBroken link to arcpy:\n\n\nUnable to Select Python 3:\n\n\nAs I read that ArcGIS Pro 1.3 uses Anaconda I tried installing that, but it was installed to a completely new location and still I had no access to arcpy from VS.  \n\nHow can I make Visual Studio see arcpy again?\n", ClosedDate=None, CommentCount=0, Creation

In [34]:
post_history.where(post_history.PostId == 203381).collect()

[Row(Comment=None, Id=643309, PostHistoryTypeId=2, PostId=203381, Text='Python/ArcPy for ArcGIS Pro 1.3 is now installed in\r\n\r\n`C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\`\r\n\r\nSo your IDE needs to be pointed at that environment to pick up the link to `arcpy`.\r\n\r\n**In Visual Studio** open the Python Environments window, click "+ Custom..." and select "Configure".\r\n\r\nGive the new environment a name e.g. *Python 64-bit 3.5 ArcPy* and populate the path fields with the path above, specifying the relevant file in that path for each field.\r\n\r\n - Prefix Path: `C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\`\r\n - Interpreter Path: `C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python.exe`\r\n - Windowed Interpreter: `C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\pythonw.exe`\r\n - Library Path: `C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\Lib`\r\n - Language Version: `3.

In [ ]:
# wszystkie z filtrem merged_posts_ids

# the `in` operator in pyspark is broken (2.2.0)
# https://issues.apache.org/jira/browse/SPARK-19701
# Funkcjonalność usunięta:
# https://github.com/apache/spark/pull/17160
#
# trzeba użyć:
# http://takwatanabe.me/pyspark/generated/generated/sql.functions.array_contains.html

#questions.filter([x for x in merged_posts_ids]).show()

# The ANTI JOIN – all values from table1 where not in table2 ????????????
# http://blog.montmere.com/2010/12/08/the-anti-join-all-values-from-table1-where-not-in-table2/

#.join(answers, questions.Id == answers.Parent) \
#                            .withColumn('TimeDelta', time_delta_udf(questions.QuestionDate, answers.AnswerDate))

In [12]:
joined_timedelta.collect()

[Row(Id=1697, AcceptedAnswerId=1723, QuestionDate=datetime.datetime(2010, 9, 3, 7, 58, 27, 23000), Tags='arcobjects python arcgis-10.0 arcmap ', AnswerId=1698, Parent=1697, AnswerDate=datetime.datetime(2010, 9, 3, 9, 28, 4, 740000), TimeDelta=5377),
 Row(Id=1697, AcceptedAnswerId=1723, QuestionDate=datetime.datetime(2010, 9, 3, 7, 58, 27, 23000), Tags='arcobjects python arcgis-10.0 arcmap ', AnswerId=1723, Parent=1697, AnswerDate=datetime.datetime(2010, 9, 4, 9, 26, 10, 723000), TimeDelta=91663),
 Row(Id=1697, AcceptedAnswerId=1723, QuestionDate=datetime.datetime(2010, 9, 3, 7, 58, 27, 23000), Tags='arcobjects python arcgis-10.0 arcmap ', AnswerId=5946, Parent=1697, AnswerDate=datetime.datetime(2011, 2, 9, 1, 3, 26, 177000), TimeDelta=13712699),
 Row(Id=2927, AcceptedAnswerId=2952, QuestionDate=datetime.datetime(2010, 10, 26, 17, 7, 25, 957000), Tags='fme ', AnswerId=2952, Parent=2927, AnswerDate=datetime.datetime(2010, 10, 26, 22, 48, 27, 823000), TimeDelta=20461),
 Row(Id=3091, Accep

In [17]:
joined_timedelta.select(min(joined_timedelta.TimeDelta)).collect()

[Row(min(TimeDelta)=-110165685)]

In [18]:
joined_timedelta.where(joined_timedelta.TimeDelta == -110165685).collect()

[Row(Id=124794, AcceptedAnswerId=None, QuestionDate=datetime.datetime(2014, 12, 7, 17, 43, 6, 980000), Tags='qgis installation mac ecw ', AnswerId=10917, Parent=124794, AnswerDate=datetime.datetime(2011, 6, 11, 16, 8, 21, 577000), TimeDelta=-110165685)]

In [19]:
posts.where(posts.Id == 124794).collect()

[Row(AcceptedAnswerId=None, AnswerCount=5, Body='I bought a imac recently and I tried to install qgis: without success! It is not possible to read the raster files.  I installed GDAL ECW plugin, but it is necessary to copy and paste another file from  ERDAS ECW/JPEG2000 following this instruction.\n\n"VERY IMPORTANT POST-INSTALL\n\nYou MUST download and install the ECW SDK yourself. I don\'t have the means to enforce export restrictions. I\'m sorry, but installation is a manual process for now:\n\nGo to http://download.intergraph.com/download-portal. Select "ERDAS ECW/JPEG2000 SDK" from the Product popup. Click on the MacOSX line, then the Download Now link.\n\nOpen the downloaded disk image and run the installer. It will install into the top of your boot drive, /Intergraph. Dig into this folder to Desktop_Read-Only/redistributable/libstdc++.\n\nCopy the libNCSEcw.dylib file to:\n\n/Library/Application Support/GDAL/1.11/Libraries\n\nand copy the whole Desktop_Read-Only/etc folder here 

197

In [ ]:

# jesli w historii jest 37, 38, to są to zmerdżowane posty
# 37 źródło, 38 cel, 38 jest starszy?

# do obliczen czasu trzeba usunac duplikaty
# ale org posty nie maja odpowiedzi?


In [35]:
post_links.where(post_links.RelatedPostId == 124794).collect()

[Row(CreationDate=datetime.datetime(2016, 4, 3, 15, 40, 54, 250000), Id=4103194, LinkTypeId=3, PostId=43830, RelatedPostId=124794),
 Row(CreationDate=datetime.datetime(2016, 4, 3, 21, 54, 56, 480000), Id=4103510, LinkTypeId=3, PostId=10914, RelatedPostId=124794)]

In [38]:
post_links.where(post_links.PostId == 43830).collect()

[Row(CreationDate=datetime.datetime(2016, 4, 3, 15, 40, 54, 250000), Id=4103194, LinkTypeId=3, PostId=43830, RelatedPostId=124794)]

In [9]:
answers.where(answers.Id == 194).collect()

[Row(Id=194, ParentId=18, CreationDate=datetime.datetime(2010, 7, 23, 12, 31, 48, 770000))]

In [ ]:
# Reputation histogram!!!

#To further study the reason behind the difference in motifs
#across forums, we plotted the distribution of users’ reputations
#in figure 7. We see that the proportion of intermediate users is
#higher in discussion based forums than in fact-based forums,
#which explains the higher representation of flat-hierarchy
#interactions (e.g. users of similar expertise level help each
#other out) in the motifs of Movies, UX, and History.

In [ ]:
# Graf kołowy: (bokeh chord)
#The graph we constructed can be succinctly described as
#an accepted answers graph. Figure 3 explains how questions
#and answers in the forum translate to the graph in our analysis.
#A node exists for each user in the forum. For each accepted
#answer, there is an edge from the asker to the answerer.
#We insert an edge into the graph only if the answer is
#accepted, unlike prior analysis on Q&A forums performed
#by other studies where there are edges for both accepted and
#non-accepted answers.